In [17]:
import numpy as np
from dti_elements import domain as Domain
from dti_elements.lasers import fuente_microscopia_1 as laser
from dti_elements.tejidos import cerebro_emb_pez_cebra as tejido

In [18]:
# Parametros de dominio

Lz = np.float32(361e-6) # 200um
Nz = 361
dz = np.float32(Lz / Nz) # 1um

Lx, Ly = np.float32(45e-6), np.float32(45e-6) # 45um x 45um
Nx, Ny = 256, 256
dx = np.float32(Lx / Nx) # 0.35um
dy = np.float32(Ly / Ny) # 0.35um

x = np.linspace(-Lx/2, Lx/2, Nx, dtype=np.float32)
y = np.linspace(-Ly/2, Ly/2, Ny, dtype=np.float32)
X, Y = np.meshgrid(x, y)

k0 = np.float32(2*np.pi / laser.wavelength)
k = np.float32(k0 * tejido.n_0)
sigma_phi = np.float32(k * tejido.Dn * tejido.l_s)
# Typical value for brain tissue (5 μm)
sigma_x = np.float32(5e-6)

domain = Domain.Domain(X, Y, Nx, Ny, Nz, dx, dy, dz, np.float32(1e-12), k0, k, sigma_phi, sigma_x)
